# Agricultural Planning Optimization with Market Coupling (2024-2030)

## Objective

Maximize 7-year total profit using elasticity-coupled demand predictions while respecting:
- Land capacity and crop compatibility
- Production-sales balance with market limits
- Rotation and diversity requirements

## Key Difference from Problem 2

**Input data**: Uses predictions from structural elasticity model (market-coupled) rather than independent forecasts.

**Model formulation**: Identical MILP structure with sales variable $y_{it}$ to handle demand constraints.

In [4]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import openpyxl

## 1. Data Loading

Load elasticity-coupled predictions and reference data.

In [5]:
stats_df = pd.read_csv('crop_predictions_problem3.csv')

# Load plot data
farmland_df = pd.read_excel('Attachment1_EN.xlsx', sheet_name='Existing Village Farmland')
farmland_df = farmland_df.dropna(subset=['Plot Name'])
farmland_df = farmland_df[farmland_df['Plot Name'].str.match(r'^[A-F]\d+$', na=False)]

# Load crop rules
crops_df = pd.read_excel('Attachment1_EN.xlsx', sheet_name='Village Crops')
crops_df = crops_df.dropna(subset=['Crop Name'])
crops_df = crops_df[crops_df['Crop ID'].notna()].head(41)

print(f"Loaded {len(stats_df)} prediction records")
print(f"Plots: {len(farmland_df)}, Crops: {len(crops_df)}")

Loaded 287 prediction records
Plots: 54, Crops: 41


## 2. Parameter Extraction

Build dictionaries for model parameters.

In [6]:
# Sets
CROPS = stats_df['Crop'].unique().tolist()
PLOTS = sorted(farmland_df['Plot Name'].tolist())
YEARS = sorted(stats_df['Year'].unique())
SEASONS = ['Single Season', 'First Season', 'Second Season']

# Plot characteristics
area = dict(zip(farmland_df['Plot Name'], farmland_df['Plot Area (Mu)']))
plot_type = dict(zip(farmland_df['Plot Name'], farmland_df['Plot Type']))

# Crop parameters (indexed by crop and year)
price = {}
cost = {}
yield_per_mu = {}
demand = {}

for _, row in stats_df.iterrows():
    i, t = row['Crop'], row['Year']
    price[i, t] = row['Price_mean']
    cost[i, t] = row['Cost_mean']
    yield_per_mu[i, t] = row['Yield_per_Mu_mean']
    demand[i, t] = row['Sales_Volume_mean']

# Crop categories
crop_type_map = dict(zip(crops_df['Crop Name'], crops_df['Crop Type']))

# Infer planting season from Planting Farmland and Notes
planting_season_map = {}
for _, row in crops_df.iterrows():
    name = row['Crop Name']
    pf = str(row.get('Planting Farmland', '')).lower()
    notes = str(row.get('Notes', '')).lower()
    
    if 'second season' in pf or 'second season' in notes:
        planting_season_map[name] = 'Second Season Only'
    else:
        planting_season_map[name] = 'Regular'

# Classify crops
grains = [c for c in CROPS if crop_type_map.get(c) in ['Grain', 'Grain (Legumes)']]
legumes = [c for c in CROPS if 'Legumes' in str(crop_type_map.get(c, ''))]
veg_regular = [c for c in CROPS if crop_type_map.get(c) in ['Vegetable', 'Vegetable (Legumes)'] and 
               planting_season_map.get(c) != 'Second Season Only']
veg_2nd_only = [c for c in CROPS if crop_type_map.get(c) in ['Vegetable', 'Vegetable (Legumes)'] and 
                planting_season_map.get(c) == 'Second Season Only']
mushrooms = [c for c in CROPS if crop_type_map.get(c) == 'Edible Mushroom']

print(f"Grains: {len(grains)}, Legumes: {len(legumes)}, Vegetables: {len(veg_regular + veg_2nd_only)}, Mushrooms: {len(mushrooms)}")

Grains: 16, Legumes: 8, Vegetables: 21, Mushrooms: 4


## 3. Compatibility Matrix

Define which crops can be planted on which plot-season combinations.

In [7]:
COMPAT = {}

for j in PLOTS:
    ptype = plot_type[j]
    
    if ptype in ['Flat Dry Land', 'Terraced Field', 'Hillside Land']:
        for i in grains + legumes:
            if i != 'Rice':  # Rice needs irrigation
                COMPAT[i, j, 'Single Season'] = 1
    
    elif ptype == 'Irrigated Land':
        if 'Rice' in CROPS:
            COMPAT['Rice', j, 'Single Season'] = 1
        for i in veg_regular:
            COMPAT[i, j, 'First Season'] = 1
        for i in veg_2nd_only:
            COMPAT[i, j, 'Second Season'] = 1
    
    elif ptype == 'Regular Greenhouse':
        for i in veg_regular:
            COMPAT[i, j, 'First Season'] = 1
        for i in mushrooms:
            COMPAT[i, j, 'Second Season'] = 1
    
    elif ptype == 'Smart Greenhouse':
        for i in veg_regular:
            COMPAT[i, j, 'First Season'] = 1
            COMPAT[i, j, 'Second Season'] = 1

VALID = [(i, j, s) for (i, j, s), v in COMPAT.items() if v == 1]
print(f"Valid crop-plot-season combinations: {len(VALID)}")

Valid crop-plot-season combinations: 796


## 4. Model Formulation

### Decision Variables

$$x_{ijst} \in \{0,1\}: \text{Plant crop } i \text{ on plot } j \text{ in season } s \text{ of year } t$$

$$y_{it} \geq 0: \text{Sales volume of crop } i \text{ in year } t$$

### Objective Function

$$\max Z = \sum_{i,t} p_{it} y_{it} - \sum_{i,j,s,t} c_{it} A_j x_{ijst}$$

### Constraints

**C1. Land capacity** (plot-season exclusivity):
$$\sum_{i} x_{ijst} \leq 1 \quad \forall j, s, t$$

**C2. Sales-production balance**:
$$y_{it} \leq \sum_{j,s} q_{it} A_j x_{ijst} \quad \forall i, t$$

**C3. Market demand limit**:
$$y_{it} \leq d_{it} \quad \forall i, t$$

**C4. Non-continuous planting**:
$$x_{ij,\text{First},t} + x_{ij,\text{Second},t} \leq 1 \quad \forall i, j, t$$

**C5. Legume rotation** (at least once every 3 years):
$$\sum_{i \in L} \sum_{s} \sum_{\tau=t}^{t+2} x_{ijs\tau} \geq 1 \quad \forall j \in J_L, t \leq 2028$$

**C6. Crop diversity** (max 20% per crop):
$$\sum_{j,s} A_j x_{ijst} \leq 0.20 \sum_{j} A_j \quad \forall i, t$$

In [14]:
# Initialize model
model = gp.Model("Problem3_Market_Coupled")
model.setParam('OutputFlag', 0)
model.setParam('TimeLimit', 300)
model.setParam('MIPGap', 0.01)

# Decision variables
x_keys = [(i, j, s, t) for (i, j, s) in VALID for t in YEARS]
x = model.addVars(x_keys, vtype=GRB.BINARY, name="plant")
y = model.addVars(CROPS, YEARS, vtype=GRB.CONTINUOUS, name="sales")

# Objective
revenue = gp.quicksum(price[i, t] * y[i, t] for i in CROPS for t in YEARS)
costs = gp.quicksum(cost[i, t] * area[j] * x[i, j, s, t] for (i, j, s, t) in x.keys())
model.setObjective(revenue - costs, GRB.MAXIMIZE)

## 5. Add Constraints

In [15]:
# C1: Land capacity
for j in PLOTS:
    for t in YEARS:
        ptype = plot_type[j]
        
        if ptype in ['Flat Dry Land', 'Terraced Field', 'Hillside Land']:
            model.addConstr(
                gp.quicksum(x[i, j, s, t] for i in CROPS for s in SEASONS if (i, j, s, t) in x) <= 1,
                name=f"land_{j}_{t}"
            )
        
        elif ptype == 'Irrigated Land':
            model.addConstr(
                gp.quicksum(x[i, j, s, t] for i in CROPS for s in ['Single Season', 'First Season'] 
                           if (i, j, s, t) in x) <= 1,
                name=f"land_first_{j}_{t}"
            )
            model.addConstr(
                gp.quicksum(x[i, j, s, t] for i in CROPS for s in ['Single Season', 'Second Season'] 
                           if (i, j, s, t) in x) <= 1,
                name=f"land_second_{j}_{t}"
            )
        
        elif ptype in ['Regular Greenhouse', 'Smart Greenhouse']:
            model.addConstr(
                gp.quicksum(x[i, j, 'First Season', t] for i in CROPS if (i, j, 'First Season', t) in x) <= 1,
                name=f"land_first_{j}_{t}"
            )
            model.addConstr(
                gp.quicksum(x[i, j, 'Second Season', t] for i in CROPS if (i, j, 'Second Season', t) in x) <= 1,
                name=f"land_second_{j}_{t}"
            )

# C2 & C3: Sales constraints
for i in CROPS:
    for t in YEARS:
        production = gp.quicksum(
            yield_per_mu[i, t] * area[j] * x[i, j, s, t]
            for j in PLOTS for s in SEASONS if (i, j, s, t) in x
        )
        model.addConstr(y[i, t] <= production, name=f"prod_{i}_{t}")
        model.addConstr(y[i, t] <= demand[i, t], name=f"demand_{i}_{t}")

# C4: Non-continuous planting
for i in CROPS:
    for j in PLOTS:
        for t in YEARS:
            if (i, j, 'First Season', t) in x and (i, j, 'Second Season', t) in x:
                model.addConstr(
                    x[i, j, 'First Season', t] + x[i, j, 'Second Season', t] <= 1,
                    name=f"noncont_{i}_{j}_{t}"
                )

# C5: Legume rotation
legume_plots = set(j for (i, j, s) in VALID if i in legumes)
for j in legume_plots:
    for t in YEARS[:-2]:
        window = [t, t + 1, t + 2]
        model.addConstr(
            gp.quicksum(x[i, j, s, tau] for i in legumes for s in SEASONS 
                       for tau in window if (i, j, s, tau) in x) >= 1,
            name=f"legume_{j}_{t}"
        )

# C6: Crop diversity (NEW - was missing in original code)
total_area = sum(area.values())
for i in CROPS:
    for t in YEARS:
        model.addConstr(
            gp.quicksum(area[j] * x[i, j, s, t] for j in PLOTS for s in SEASONS 
                       if (i, j, s, t) in x) <= 0.20 * total_area,
            name=f"diversity_{i}_{t}"
        )

## 6. Solve Model

In [10]:
model.optimize()

if model.Status == GRB.OPTIMAL:
    print(f"Status: Optimal")
    print(f"Total 7-year profit: ¥{model.objVal:,.2f}")
    print(f"Average annual profit: ¥{model.objVal / 7:,.2f}")
elif model.Status == GRB.TIME_LIMIT:
    print(f"Time limit reached")
    print(f"Best solution found: ¥{model.objVal:,.2f}")
    print(f"Gap: {model.MIPGap * 100:.2f}%")
else:
    print(f"No solution found (Status code: {model.Status})")

Status: Optimal
Total 7-year profit: ¥100,229,466.07
Average annual profit: ¥14,318,495.15


## 7. Extract Results

In [11]:
if model.Status in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
    results = []
    for (i, j, s, t) in x.keys():
        if x[i, j, s, t].X > 0.5:
            results.append({
                'Crop': i,
                'Plot': j,
                'Year': t,
                'Season': s,
                'Area_Mu': area[j]
            })
    
    results_df = pd.DataFrame(results)
    print(f"Total planting decisions: {len(results_df)}")

Total planting decisions: 326


## 8. Validation

In [12]:
if 'results_df' in locals() and not results_df.empty:
    
    # Check rotation constraint
    for j in legume_plots:
        for t in YEARS[:-2]:
            window = [t, t + 1, t + 2]
            legume_count = len(results_df[
                (results_df['Plot'] == j) & 
                (results_df['Year'].isin(window)) & 
                (results_df['Crop'].isin(legumes))
            ])
            if legume_count == 0:
                print(f"Plot {j} has no legumes in {window}")
    
    # Check diversity constraint
    for t in YEARS:
        year_data = results_df[results_df['Year'] == t]
        crop_areas = year_data.groupby('Crop')['Area_Mu'].sum()
        violations = crop_areas[crop_areas > 0.20 * total_area]
        if not violations.empty:
            for crop, area_used in violations.items():
                print(f"Year {t}: {crop} occupies {area_used / total_area:.1%} > 20%")
    
    # Check non-continuous planting
    for t in YEARS:
        year_data = results_df[results_df['Year'] == t]
        for j in PLOTS:
            plot_data = year_data[year_data['Plot'] == j]
            for crop in plot_data['Crop'].unique():
                crop_seasons = plot_data[plot_data['Crop'] == crop]['Season'].tolist()
                if 'First Season' in crop_seasons and 'Second Season' in crop_seasons:
                    print(f"{crop} planted in both seasons on {j} in {t}")
    
    print("\nValidation complete")


Validation complete


## 9. Export to Excel Template

In [13]:
if 'results_df' in locals() and not results_df.empty:
    # Load template
    template_path = 'result2_EN.xlsx'
    wb_template = openpyxl.load_workbook(template_path)
    ws_template = wb_template['2024']
    
    # Get crop columns
    crop_columns = []
    for col_idx in range(3, ws_template.max_column + 1):
        crop_name = ws_template.cell(1, col_idx).value
        if crop_name:
            crop_columns.append(crop_name)
    
    # Get plot lists
    season1_plots = []
    for row_idx in range(2, 56):
        plot = ws_template.cell(row_idx, 2).value
        if plot:
            season1_plots.append(plot)
    
    # Season 2 plots: only D, E, F plots (FIXED)
    season2_plots = [p for p in PLOTS if p.startswith(('D', 'E', 'F'))]
    
    # Create output file
    output_path = 'result3_EN_OUTPUT.xlsx'
    wb_output = openpyxl.load_workbook(template_path)
    
    for year in YEARS:
        ws = wb_output[str(year)]
        year_data = results_df[results_df['Year'] == year]
        
        # Initialize all crop cells to 0
        for row_idx in range(2, 84):
            for crop_idx, crop in enumerate(crop_columns):
                col_idx = crop_idx + 3
                ws.cell(row_idx, col_idx).value = 0
        
        # Fill Season 1 (rows 2-55)
        for idx, plot in enumerate(season1_plots):
            row_idx = 2 + idx
            
            plot_s1 = year_data[
                (year_data['Plot'] == plot) & 
                (year_data['Season'].isin(['Single Season', 'First Season']))
            ]
            
            for _, p in plot_s1.iterrows():
                crop = p['Crop']
                if crop in crop_columns:
                    col_idx = crop_columns.index(crop) + 3
                    ws.cell(row_idx, col_idx).value = p['Area_Mu']
        
        # Fill Season 2 (rows 56-83) - FIXED to use correct plots
        for idx, plot in enumerate(season2_plots):
            row_idx = 56 + idx
            
            plot_s2 = year_data[
                (year_data['Plot'] == plot) & 
                (year_data['Season'] == 'Second Season')
            ]
            
            for _, p in plot_s2.iterrows():
                crop = p['Crop']
                if crop in crop_columns:
                    col_idx = crop_columns.index(crop) + 3
                    ws.cell(row_idx, col_idx).value = p['Area_Mu']
    
    wb_output.save(output_path)
    print(f"Results exported to: {output_path}")

Results exported to: result3_EN_OUTPUT.xlsx
